In [5]:
# Activity 3: Data Standardization & Validation

# Task A: Enforcing Data Formats & Constraints

# 13. Date Format Standardization:
# - Convert all date entries into a uniform format (e.g., YYYY-MM-DD).





# 14. Numeric Constraints Enforcement:
# - Check and enforce numeric constraints (e.g., age > 0).






# 15. String Format Checks:
# - Ensure text fields meet certain constraints (e.g., valid email format).
import pandas as pd
import numpy as np
import re
from datetime import datetime

def standardize_and_validate(df):
    """
    Perform data standardization and validation on a DataFrame
    
    Parameters:
        df (pd.DataFrame): Input dataframe
        
    Returns:
        pd.DataFrame: Standardized and validated dataframe
        pd.DataFrame: Report of validation issues
    """
    # Make a copy of the original dataframe
    df_clean = df.copy()
    validation_report = pd.DataFrame(columns=['column', 'issue', 'count'])
    
    # 13. Date Format Standardization
    date_columns = [col for col in df.columns if 'date' in col.lower() or 'join' in col.lower()]
    for col in date_columns:
        try:
            df_clean[col] = pd.to_datetime(df_clean[col]).dt.strftime('%Y-%m-%d')
            print(f"Standardized date format for column: {col}")
        except Exception as e:
            invalid_count = df_clean[col].isna().sum()
            if invalid_count > 0:
                validation_report = validation_report.append({
                    'column': col,
                    'issue': 'Invalid date format',
                    'count': invalid_count
                }, ignore_index=True)
            print(f"Date standardization failed for {col}: {str(e)}")
    
    # 14. Numeric Constraints Enforcement
    numeric_constraints = {
        'age': {'min': 0, 'max': 120},
        'salary': {'min': 0},
        'quantity': {'min': 0, 'integer': True}
    }
    
    for col, constraints in numeric_constraints.items():
        if col in df_clean.columns:
            # Check min/max constraints
            if 'min' in constraints:
                invalid = df_clean[col] < constraints['min']
                invalid_count = invalid.sum()
                if invalid_count > 0:
                    validation_report = validation_report.append({
                        'column': col,
                        'issue': f"Value < {constraints['min']}",
                        'count': invalid_count
                    }, ignore_index=True)
                    df_clean.loc[invalid, col] = np.nan
            
            if 'max' in constraints:
                invalid = df_clean[col] > constraints['max']
                invalid_count = invalid.sum()
                if invalid_count > 0:
                    validation_report = validation_report.append({
                        'column': col,
                        'issue': f"Value > {constraints['max']}",
                        'count': invalid_count
                    }, ignore_index=True)
                    df_clean.loc[invalid, col] = np.nan
            
            # Check integer constraint
            if constraints.get('integer', False):
                non_integer = df_clean[col].dropna().apply(lambda x: not x.is_integer())
                invalid_count = non_integer.sum()
                if invalid_count > 0:
                    validation_report = validation_report.append({
                        'column': col,
                        'issue': 'Non-integer value',
                        'count': invalid_count
                    }, ignore_index=True)
    
    # 15. String Format Checks
    string_checks = {
        'email': r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$',
        'phone': r'^\+?[\d\s\-\(\)]{7,}$',
        'zip_code': r'^\d{5}(-\d{4})?$'
    }
    
    for col, pattern in string_checks.items():
        if col in df_clean.columns:
            invalid = ~df_clean[col].astype(str).str.match(pattern, na=True)
            invalid_count = invalid.sum()
            if invalid_count > 0:
                validation_report = validation_report.append({
                    'column': col,
                    'issue': f'Invalid {col} format',
                    'count': invalid_count
                }, ignore_index=True)
                df_clean.loc[invalid, col] = np.nan
    
    return df_clean, validation_report

# Example Usage
if __name__ == "__main__":
    # Create sample data
    data = {
        'customer_id': [1, 2, 3, 4, 5],
        'name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
        'email': ['alice@example.com', 'bob@test', 'charlie@example.com', 'david@test.org', 'invalid-email'],
        'phone': ['(123) 456-7890', '123-456-7890', 'invalid', '+1 234 567 8901', '5551234'],
        'join_date': ['01/15/2023', '2023-02-01', '15-03-2023', '04/01/23', '2023-05-15'],
        'age': [25, -5, 30, 150, 42],
        'salary': [50000, 60000, -1000, 75000, 80000],
        'quantity': [10, 5.5, 3, 8, 2.1],
        'zip_code': ['12345', '12345-6789', 'invalid', '98765', '1234']
    }
    df = pd.DataFrame(data)
    
    print("Original Data:")
    print(df)
    
    # Perform standardization and validation
    df_clean, report = standardize_and_validate(df)
    
    print("\nStandardized and Validated Data:")
    print(df_clean)
    
    print("\nValidation Issues Found:")
    if not report.empty:
        print(report)
    else:
        print("No validation issues found!")
    
    # Save cleaned data
    df_clean.to_csv('cleaned_data.csv', index=False)
    print("\nSaved cleaned data to 'cleaned_data.csv'")

Original Data:
   customer_id     name                email            phone   join_date  \
0            1    Alice    alice@example.com   (123) 456-7890  01/15/2023   
1            2      Bob             bob@test     123-456-7890  2023-02-01   
2            3  Charlie  charlie@example.com          invalid  15-03-2023   
3            4    David       david@test.org  +1 234 567 8901    04/01/23   
4            5      Eve        invalid-email          5551234  2023-05-15   

   age  salary  quantity    zip_code  
0   25   50000      10.0       12345  
1   -5   60000       5.5  12345-6789  
2   30   -1000       3.0     invalid  
3  150   75000       8.0       98765  
4   42   80000       2.1        1234  
Date standardization failed for join_date: time data "2023-02-01" doesn't match format "%m/%d/%Y", at position 1. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily i

AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
# Task B: Addressing Inconsistent Representations

# 16. Standardizing Date Formats:
# - Identify and correct inconsistent date formats within the dataset.








# 17. Pattern Matching for Consistency:
# - Standardize phone numbers to a specific pattern (e.g., (123) 456-7890).





# 18. Handling Mixed Case Text:
# - Convert all text entries to a consistent case (e.g., all uppercase).

import pandas as pd
import re
from datetime import datetime

def standardize_data(df):
    """
    Standardize inconsistent data representations in a DataFrame
    
    Parameters:
        df (pd.DataFrame): Input dataframe
        
    Returns:
        pd.DataFrame: Standardized dataframe
        dict: Summary of standardization operations
    """
    # Make a copy of the original dataframe
    df_clean = df.copy()
    standardization_report = {}
    
    # 16. Standardizing Date Formats
    date_columns = [col for col in df.columns if 'date' in col.lower()]
    standardization_report['date_columns'] = {}
    
    for col in date_columns:
        original_values = df_clean[col].astype(str).unique()[:5]  # Sample of original values
        try:
            # Try to parse dates automatically first
            df_clean[col] = pd.to_datetime(df_clean[col], errors='raise')
            df_clean[col] = df_clean[col].dt.strftime('%Y-%m-%d')
            standardization_report['date_columns'][col] = {
                'action': 'Standardized to YYYY-MM-DD',
                'sample_original': list(original_values),
                'sample_converted': df_clean[col].unique()[:5]
            }
        except Exception as e:
            # Fallback to manual parsing if automatic fails
            try:
                df_clean[col] = df_clean[col].apply(lambda x: parse_custom_date(x) if pd.notna(x) else x)
                standardization_report['date_columns'][col] = {
                    'action': 'Custom parsing applied',
                    'sample_original': list(original_values),
                    'sample_converted': df_clean[col].unique()[:5]
                }
            except:
                standardization_report['date_columns'][col] = {
                    'action': 'Failed to standardize',
                    'error': str(e),
                    'sample_original': list(original_values)
                }
    
    # 17. Pattern Matching for Phone Numbers
    phone_columns = [col for col in df.columns if 'phone' in col.lower()]
    standardization_report['phone_columns'] = {}
    
    for col in phone_columns:
        original_values = df_clean[col].astype(str).unique()[:5]
        df_clean[col] = df_clean[col].apply(standardize_phone_number)
        standardization_report['phone_columns'][col] = {
            'action': 'Standardized to (XXX) XXX-XXXX format',
            'sample_original': list(original_values),
            'sample_converted': df_clean[col].unique()[:5]
        }
    
    # 18. Handling Mixed Case Text
    text_columns = [col for col in df.columns if df[col].dtype == 'object' and 
                   col not in date_columns and col not in phone_columns]
    standardization_report['text_columns'] = {}
    
    for col in text_columns:
        original_values = df_clean[col].astype(str).unique()[:5]
        df_clean[col] = df_clean[col].str.upper()
        standardization_report['text_columns'][col] = {
            'action': 'Converted to uppercase',
            'sample_original': list(original_values),
            'sample_converted': df_clean[col].unique()[:5]
        }
    
    return df_clean, standardization_report

def parse_custom_date(date_str):
    """Handle custom date formats that pandas can't automatically parse"""
    if pd.isna(date_str):
        return date_str
    
    # Try common date formats
    for fmt in ('%m/%d/%Y', '%d/%m/%Y', '%Y-%m-%d', '%m-%d-%Y', '%d-%m-%Y', 
                '%b %d, %Y', '%d %b %Y', '%B %d, %Y', '%d %B %Y'):
        try:
            return datetime.strptime(str(date_str), fmt).strftime('%Y-%m-%d')
        except ValueError:
            continue
    
    # If all formats fail, return original (will be marked as invalid)
    return date_str

def standardize_phone_number(phone):
    """Standardize phone numbers to (XXX) XXX-XXXX format"""
    if pd.isna(phone):
        return phone
    
    phone_str = str(phone)
    # Remove all non-digit characters
    digits = re.sub(r'[^\d]', '', phone_str)
    
    # Format based on length
    if len(digits) == 10:
        return f"({digits[:3]}) {digits[3:6]}-{digits[6:]}"
    elif len(digits) == 11 and digits[0] == '1':  # US country code
        return f"+1 ({digits[1:4]}) {digits[4:7]}-{digits[7:]}"
    else:
        return phone_str  # Return original if can't standardize

# Example Usage
if __name__ == "__main__":
    # Create sample data with inconsistent representations
    data = {
        'customer_id': [1, 2, 3, 4, 5],
        'name': ['alice', 'BOB', 'Charlie', 'dAvId', 'EVE'],
        'email': ['ALICE@test.com', 'bob@TEST.com', 'CHARLIE@example.COM', 'david@test.org', 'eve@EXAMPLE.org'],
        'phone': ['123-456-7890', '(123) 456-7890', '123.456.7890', '1234567890', '+1 (234) 567-8901'],
        'join_date': ['01/15/2023', '2023-02-01', '15-03-2023', 'Apr 1, 2023', '2023-May-15'],
        'last_purchase_date': ['1/1/23', '02/15/2023', 'March 5, 2023', '04-01-23', '05/15/2023']
    }
    df = pd.DataFrame(data)
    
    print("Original Data:")
    print(df)
    
    # Perform standardization
    df_clean, report = standardize_data(df)
    
    print("\nStandardized Data:")
    print(df_clean)
    
    print("\nStandardization Report:")
    for section, details in report.items():
        print(f"\n=== {section.replace('_', ' ').title()} ===")
        for col, info in details.items():
            print(f"\nColumn: {col}")
            print(f"Action: {info['action']}")
            print("Original samples:", info['sample_original'])
            if 'sample_converted' in info:
                print("Converted samples:", info['sample_converted'])
            if 'error' in info:
                print("Error:", info['error'])
    
    # Save cleaned data
    df_clean.to_csv('standardized_data.csv', index=False)
    print("\nSaved standardized data to 'standardized_data.csv'")










Original Data:
   customer_id     name                email              phone    join_date  \
0            1    alice       ALICE@test.com       123-456-7890   01/15/2023   
1            2      BOB         bob@TEST.com     (123) 456-7890   2023-02-01   
2            3  Charlie  CHARLIE@example.COM       123.456.7890   15-03-2023   
3            4    dAvId       david@test.org         1234567890  Apr 1, 2023   
4            5      EVE      eve@EXAMPLE.org  +1 (234) 567-8901  2023-May-15   

  last_purchase_date  
0             1/1/23  
1         02/15/2023  
2      March 5, 2023  
3           04-01-23  
4         05/15/2023  

Standardized Data:
   customer_id     name                email              phone    join_date  \
0            1    ALICE       ALICE@TEST.COM     (123) 456-7890   2023-01-15   
1            2      BOB         BOB@TEST.COM     (123) 456-7890   2023-02-01   
2            3  CHARLIE  CHARLIE@EXAMPLE.COM     (123) 456-7890   2023-03-15   
3            4    DAVID   

/tmp/ipykernel_15712/558802915.py:50: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_clean[col] = pd.to_datetime(df_clean[col], errors='raise')
